<a href="https://colab.research.google.com/github/Rumeysakeskin/YOLO-Darknet-Video-Image-Detection-Traffic-Signs/blob/main/traffic_sign_detection_on_video.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import cv2
import numpy as np
import time
import matplotlib.pyplot as plt

**LOAD YOLOV3 NETWORK**

In [3]:
with open('classes.names') as f:
    labels = [line.strip() for line in f]

# Loading trained YOLO v3 Objects Detector
# with the help of 'dnn' library from OpenCV
network = cv2.dnn.readNetFromDarknet('yolov3_test.cfg',
                                      'yolov3_train_3000.weights')

# Getting list with names of all layers from YOLO v3 network
layers_names_all = network.getLayerNames()

# Getting only output layers' names that we need from YOLO v3 algorithm with function that returns indexes of layers with unconnected outputs
layers_names_output = [layers_names_all[i - 1] for i in network.getUnconnectedOutLayers()]

print(layers_names_output)  # Detections at layers: ['yolo_82', 'yolo_94', 'yolo_106']

# Setting minimum probability to eliminate weak predictions
probability_minimum = 0.5

# Setting threshold for filtering weak bounding boxes with non-maximum suppression
threshold = 0.3

# Generating colours for representing every detected object with function randint(low, high=None, size=None, dtype='l')
colours = np.random.randint(0, 255, size=(len(labels), 3), dtype='uint8')

['yolo_82', 'yolo_94', 'yolo_106']


**OBJECT DETECTION ON VIDEO**

In [10]:
capture = cv2.VideoCapture("/content/traffic-sign-test.mp4") 
ret = True
# Preparing variable for writer that we will use to write processed frames
writer= None
# Preparing variables for spatial dimensions of the frames
h, w = None, None
# Defining variable for counting frames at the end we will show total amount of processed frames
f = 0
# Defining variable for counting total time at the end we will show time spent for processing all frames
t = 0

while ret: # Break the loop if the video has ended
    ret, frame = capture.read()
    if not ret:
      break
    if w is None or h is None:  
      h, w = frame.shape[:2]

    # blob = cv2.dnn.blobFromImage(image, scalefactor=1.0, size, mean, swapRB=True)     
    blob = cv2.dnn.blobFromImage(frame, 1 / 255.0, (416, 416), swapRB=True, crop=False)
    network.setInput(blob)  # setting blob as input to the network
    start = time.time()
    output_from_network = network.forward(layers_names_output)
    end = time.time()

    f += 1
    t += end - start

    # Preparing lists for detected bounding boxes, obtained confidences and class's number
    bounding_boxes = []
    confidences = []
    class_numbers = []

    # Going through all output layers after feed forward pass
    for result in output_from_network:
        # Going through all detections from current output layer
        for detected_objects in result:
            # Getting 4 classes' probabilities for current detected object
            scores = detected_objects[5:]
            # Getting index of the class with the maximum value of probability
            class_current = np.argmax(scores)
            # Getting value of probability for defined class
            confidence_current = scores[class_current]

            # Eliminating weak predictions with minimum probability
            if confidence_current > probability_minimum:
                # Scaling bounding box coordinates to the initial image size
                # YOLO data format keeps coordinates for center of bounding box
                # and its current width and height
                # That is why we can just multiply them elementwise
                # to the width and height
                # of the original image and in this way get coordinates for center
                # of bounding box, its width and height for original image
                box_current = detected_objects[0:4] * np.array([w, h, w, h])

                # Now, from YOLO data format, we can get top left corner coordinates
                # that are x_min and y_min
                x_center, y_center, box_width, box_height = box_current
                x_min = int(x_center - (box_width / 2))
                y_min = int(y_center - (box_height / 2))

                # Adding results into prepared lists
                bounding_boxes.append([x_min, y_min, int(box_width), int(box_height)])
                confidences.append(float(confidence_current))
                class_numbers.append(class_current)

    results = cv2.dnn.NMSBoxes(bounding_boxes, confidences, probability_minimum, threshold)

    counter = 1

    # Checking if there is at least one detected object after non-maximum suppression
    if len(results) > 0:
        # Going through indexes of results
        for i in results.flatten():
            # Showing labels of the detected objects

            # Incrementing counter
            counter += 1

            # Getting current bounding box coordinates,
            # its width and height
            x_min, y_min = bounding_boxes[i][0], bounding_boxes[i][1]
            box_width, box_height = bounding_boxes[i][2], bounding_boxes[i][3]

            # Preparing colour for current bounding box
            # and converting from numpy array to list
            colour_box_current = colours[class_numbers[i]].tolist()

            # Drawing bounding box on the original image
            cv2.rectangle(frame, (x_min, y_min),
                          (x_min + box_width, y_min + box_height),
                          colour_box_current, 2)

            # Preparing text with label and confidence for current bounding box
            text_box_current = '{}: {:.4f}'.format(labels[int(class_numbers[i])],
                                                    confidences[i])

            # Putting text with label and confidence on the original image
            cv2.putText(frame, text_box_current, (x_min, y_min - 3),
                        cv2.FONT_HERSHEY_COMPLEX, 0.5, colour_box_current, 1)
            

    # Writing processed frame into the file
    # Initializing writer
    # we do it only once from the very beginning when we get spatial dimensions of the frames
    if writer is None:
        # Constructing code of the codec to be used in the function VideoWriter
        fourcc = cv2.VideoWriter_fourcc(*'mp4v')

        # Writing current processed frame into the video file
        writer = cv2.VideoWriter('result_traffic_sign.mp4', fourcc, 30,
                                 (frame.shape[1], frame.shape[0]), True)

    # Write processed current frame to the file
    writer.write(frame)


print()
print('Total number of frames', f)
print('Total amount of time {:.5f} seconds'.format(t))

# Releasing video reader and writer
capture.release()
writer.release()

"""
Parameters for cv2.VideoWriter():
    filename - Name of the output video file.
    fourcc - 4-character code of codec used to compress the frames, colour or pixel format used in media files.
    fps	- Frame rate of the created video.
    frameSize - Size of the video frames.
    isColor	- If it True, the encoder will expect and encode colour frames.
"""


Total number of frames 56
Total amount of time 70.96405 seconds


'\nParameters for cv2.VideoWriter():\n    filename - Name of the output video file.\n    fourcc - 4-character code of codec used to compress the frames, colour or pixel format used in media files.\n    fps\t- Frame rate of the created video.\n    frameSize - Size of the video frames.\n    isColor\t- If it True, the encoder will expect and encode colour frames.\n'